In [1]:
import cv2
import time
import math
import csv
import pandas as pd
from pathlib import Path
import re
import numpy as np
import sys
import os
import ipyparallel as ipp
from ipyparallel import Client, parallel, require

import mediapipe as mp

In [2]:
class poseDetector():

    def __init__(self, mode=False, upBody=False, smooth=True,
                 detectionCon=0.5, trackCon=0.5):

        self.mode = mode

        self.upBody = upBody

        self.smooth = smooth

        self.detectionCon = detectionCon

        self.trackCon = trackCon

        self.mpDraw = mp.solutions.drawing_utils

        self.mpPose = mp.solutions.pose
# test diff values of this param for robustness, maybe always remove first and last peaks
        self.pose =self.mpPose.Pose(model_complexity=2,smooth_landmarks = True,min_detection_confidence=0.5,
                          min_tracking_confidence=0.5)# self.mpPose.Pose(~self.mode, self.upBody, self.smooth,     self.detectionCon, self.trackCon~)

    def findPose(self, img, draw=True):   # THIS draw is responsible for drawing stick figure

        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        self.results = self.pose.process(imgRGB)
        # print(self.results)
        if self.results.pose_landmarks:
            if draw:
                # print(self.results.pose_landmarks,self.mpPose.POSE_CONNECTIONS)
                self.mpDraw.draw_landmarks(img, self.results.pose_landmarks,
                                           self.mpPose.POSE_CONNECTIONS)
        # print("pose worls landmarks")
        # self.mpDraw.plot_landmarks(
        # self.results.pose_world_landmarks, self.mpPose.POSE_CONNECTIONS)
        return img

    def findPosition(self, img, draw=True):
        font                   = cv2.FONT_HERSHEY_SIMPLEX
        fontScale              = .5
        fontColor              = (255,255,255)
        thickness              = 2
        lineType               = 2
        self.lmList = []

        if self.results.pose_landmarks:

            for id, lm in enumerate(self.results.pose_landmarks.landmark):

                h, w, c = img.shape
                
                if lm.visibility <.3:
                    lm.x=lm.y=lm.z=-1
#                 print(id, " = ",lm.visibility)
                # if id == 25:
                #     print(id, lm)
                cx, cy, cz = int(lm.x * w), int(lm.y * h), int(lm.z * w)

                self.lmList.append([id, lm.x, lm.y, lm.z])

                if draw:
                    cv2.putText(img,str(id),    (cx,cy),     font,     fontScale,    fontColor,    thickness,    lineType)
                    cv2.circle(img, (cx, cy), 5, (255, 0, 0), cv2.FILLED)
        return self.lmList
detector=poseDetector()

In [ ]:
tic=time.time()
frame=cv2.imread('./TestAnalysis/Capture.jpg')
detector = poseDetector()
img = detector.findPose(frame)
cv2.imshow("",img)
cv2.waitKey(0)
lmList = detector.findPosition(img, draw=True)
print(time.time()-tic)

for value in lmList:
    if(value[1]==-1):
        print(value[0])

## Frame Extraction

In [ ]:
cap=cv2.VideoCapture('T1.mp4')
print(cap.get(cv2.CAP_PROP_FRAME_COUNT))
cap.get(cv2.CAP_PROP_FPS)

In [ ]:
tic=time.time()
i=0
cap = cv2.VideoCapture('T2.mp4')
while True:
    ret, frame = cap.read()
    if not ret:
        break
    i += 1
    cv2.imwrite(f'./TestAnalysis/T2/frame_{i}.jpg', frame)
cap.release()
print(time.time()-tic)

## Frame Analysis

In [6]:
tic=time.time()
folder_path = './TestAnalysis/T1'
file_list =(os.listdir(folder_path))
sorted_file_list = sorted(file_list, key=lambda x: int(re.search(r'\d+', x).group()))

t1_frames=[]
for frame in sorted_file_list[:1000]:
    frame=cv2.imread('./TestAnalysis/T1/'+frame)
    t1_frames.append(frame)
print(len(t1_frames))
print(time.time()-tic)

1000
35.632554054260254


In [7]:
def isInteresting(frame):
    
    # Load YOLOv3 network
    net = cv2.dnn.readNetFromDarknet('yolov3.cfg', 'yolov3.weights')
    # Get the output layer names
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
    # Load the image
    image = frame
    # Resize and normalize the image
    blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (416, 416), swapRB=True, crop=False)
    # Set the blob as input to the network
    net.setInput(blob)
    # Perform forward pass and get output
    outputs = net.forward(output_layers)
    # Initialize lists for bounding boxes, confidences, and class IDs
    boxes = []
    confidences = []
    class_ids = []
    # Set confidence threshold and non-maximum suppression threshold
    conf_threshold = 0.5
    nms_threshold = 0.4
    # Iterate over each output layer
    for output in outputs:
        # Iterate over each detection
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            # Filter detections by class (person)
            if class_id == 0 and confidence > conf_threshold:
                # Scale the bounding box coordinates to the original image size
                width, height = image.shape[1], image.shape[0]
                x, y, w, h = detection[:4] * np.array([width, height, width, height])
                x_min, y_min = int(x - w / 2), int(y - h / 2)
                # Add bounding box, confidence, and class ID to respective lists
                boxes.append([x_min, y_min, int(w), int(h)])
                confidences.append(float(confidence))
                class_ids.append(class_id)
    # Apply non-maximum suppression to eliminate overlapping boxes
    indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)
    b=False
    num_persons=len(indices)
#     print('boxes: ',(boxes))
#     for i in range(len(indices)):
#         x1, y1, w1, h1 = boxes[indices[i]]
#         label = f"Person {i+1}"
#         cv2.rectangle(image, (x1, y1), (x1 + w1, y1 + h1), (255, 0, 0), 2)
#         cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)
#     cv2.imwrite('./check.jpg',frame)
    for i in range(len(indices)):
        if(b):
            break
        x1, y1, w1, h1 = boxes[indices[i]]
        center_x1 = x1 + w1 / 2
        center_y1 = y1 + h1 / 2    
        for j in range(i+1, len(indices)):
            x2, y2, w2, h2 = boxes[indices[j]]
            # Calculate the center coordinates of box j
            center_x2 = x2 + w2 / 2
            center_y2 = y2 + h2 / 2
            # Calculate the distances between the centers of box i and box j
            x_distance = abs(center_x1 - center_x2)
            y_distance = abs(center_y1 - center_y2)
#             print(f"Distance: Person {i+1} and Person {j+1}: {x_distance}, {y_distance}")
            if(x_distance<22 and y_distance>=300 and y_distance<=400):
                cv2.rectangle(image, (x1, y1), (x1 + w1, y1 + h1), (255, 0, 0), 2)
                cv2.rectangle(image, (x2, y2), (x2 + w2, y2 + h2), (255, 0, 0), 2)
                b=True
                break
    if(b and num_persons>=4 and num_persons<=9):
        return True
    return False

In [ ]:
tic=time.time()
i=0
idx=0
while(idx<500):
    i+=1
    frame=t1_frames[idx]
    if(isInteresting(frame)):
        for k in range(30):
            cv2.imwrite(f'./TestAnalysis/Check/frame_{i}.jpg',frame)
            i+=1
            idx+=1
            frame=t1_frames[idx]
    idx+=1
print('Time to store frames: ',time.time()-tic)

In [ ]:
folder_path = './TestAnalysis/cropped/'
file_list =(os.listdir(folder_path))
sorted_file_list = sorted(file_list, key=lambda x: int(re.search(r'\d+', x).group()))
crop_frames=[]
for frame in sorted_file_list:
    f=cv2.imread(folder_path+frame)
    crop_frames.append(f)
print(len(crop_frames))

In [ ]:
dic={}
i=0
for frame in check_frames[:50]:
    h,_,_=frame.shape
    mid=h//2
    top=frame[:mid,:] # saving particular width frame
    cv2.imshow('',top)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    booli=input()
    if(booli=='y'):
        detector=poseDetector()
        img = detector.findPose(top)
        lmList = detector.findPosition(img, draw=True)
        cv2.imshow('',img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        normalized_vectors = []
        for pt in lmList:
            normalized_vectors.append(( pt[1], pt[2], pt[3]))
        dic[i] = normalized_vectors
        i+=1
    elif(booli=='n'):
        continue
    else:
        break
#     detector = poseDetector()
#     img = detector.findPose(frame)
#     cv2.imshow('',img)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
#     lmList = detector.findPosition(img, draw=True)
#     normalized_vectors = []
#     for pt in lmList:
#         normalized_vectors.append(( pt[1], pt[2], pt[3]))
#     dic[i] = normalized_vectors
#     i = i + 1 


In [6]:
cv2.destroyAllWindows()

In [ ]:
tic=time.time()
i=0
dic={}
for frame in crop_frames:
    detector=poseDetector()
    img = detector.findPose(frame)
    lmList = detector.findPosition(img, draw=True)
    cv2.imshow('',img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    normalized_vectors = []
    for pt in lmList:
        normalized_vectors.append(( pt[1], pt[2], pt[3]))
    dic[i] = normalized_vectors
    i+=1

print(time.time()-tic)

## Getting coordinates

In [19]:
cap=cv2.VideoCapture('t2.webm')
# Callback function for mouse events
def mouse_callback(event, x, y, flags, param):
    if event == cv2.EVENT_RBUTTONDOWN:
        print(f"Coordinates: ({x}, {y})")

# Create a named window and set the callback function
cv2.namedWindow("Frame")
# cv2.resizeWindow("Frame", 800, 600)

cv2.setMouseCallback("Frame", mouse_callback)
# cv2.imshow("Frame", frame)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [5]:
frames=[]
while True:
    ret, frame = cap.read()
    frames.append(frame)
    if(len(frames)>100):
        break
cap.release()
i=0
for frame in frames[96:]:
    cv2.imwrite(f'./{i}.jpg',frame)
    i+=1
cv2.destroyAllWindows()

In [3]:
folder_path = './TestAnalysis/Check/'
file_list =(os.listdir(folder_path))
sorted_file_list = sorted(file_list, key=lambda x: int(re.search(r'\d+', x).group()))
frames=[]

top_left = (500, 100)
top_right = (1026, 90)
bottom_right = (751, 500)
bottom_left = (774, 345)

for file in sorted_file_list:
    frame=cv2.imread(folder_path+file)
    cropped_image = frame[top_left[1]:bottom_right[1], top_left[0]:bottom_right[0]]
    frames.append(cropped_image)
#     cv2.imshow('Frame',cropped_image)
#     cv2.waitKey(0)
#     cv2.imwrite('./Cropped_Image.jpg', cropped_image)
print('frame length:',len(frames))
cv2.destroyAllWindows()


frame length: 150


In [5]:
dic={}
i=0
for frame in frames:
    img = detector.findPose(frame)
    lmList = detector.findPosition(img, draw=True)
    cv2.imshow('',img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    normalized_vectors = []
    for pt in lmList:
        normalized_vectors.append(( pt[1], pt[2], pt[3]))
    dic[i]=normalized_vectors